# Least Square

In [4]:
import sys
my_path = r'/home/ilaria/Scrivania/Machine_Learning/Project_1/Project1_ML'
sys.path.insert(0,my_path + r'/code/COMMON')

import numpy as np 
import matplotlib.pyplot as plt
from proj1_helpers import * 
from implementations import *
from outliers import handle_outliers
from labels import idx_2labels
from standard import standardize
from costs import compute_loglikelihood_reg
from sigmoid import * 
from extend_features import *

In [5]:
yb, input_data, ids = load_csv_data(my_path + r'/data/train.csv', sub_sample=False)
print('Data loaded!')
print(input_data.shape)

Data loaded!
(250000, 30)


# Handle -999

In [6]:
input_data, Y = handle_outliers(input_data,yb,-999, 'mean') # substiution with mean because the standardization
                                                           #can be affected, otherwise we should delete the whole row
ind_back, ind_sig = idx_2labels(Y, [-1,1])
Y[ind_back] = 0

# get feature names 
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
log = True
degree = 1
X0, features = extend_features(input_data, names, degree, log)

-999 are replaced by the mean value of the feature
---------------------------
Features have been set to the power(s): [1]
16 Features of the momentum have been added
4 logarithmic features have been added.
Data have been standardized.
---------------------------


In [7]:
X0.shape

(250000, 50)

In [8]:
features

[(0, 'DER_mass_MMC_power_1'),
 (1, 'DER_mass_transverse_met_lep_power_1'),
 (2, 'DER_mass_vis_power_1'),
 (3, 'DER_pt_h_power_1'),
 (4, 'DER_deltaeta_jet_jet_power_1'),
 (5, 'DER_mass_jet_jet_power_1'),
 (6, 'DER_prodeta_jet_jet_power_1'),
 (7, 'DER_deltar_tau_lep_power_1'),
 (8, 'DER_pt_tot_power_1'),
 (9, 'DER_sum_pt_power_1'),
 (10, 'DER_pt_ratio_lep_tau_power_1'),
 (11, 'DER_met_phi_centrality_power_1'),
 (12, 'DER_lep_eta_centrality_power_1'),
 (13, 'PRI_tau_pt_power_1'),
 (14, 'PRI_tau_eta_power_1'),
 (15, 'PRI_tau_phi_power_1'),
 (16, 'PRI_lep_pt_power_1'),
 (17, 'PRI_lep_eta_power_1'),
 (18, 'PRI_lep_phi_power_1'),
 (19, 'PRI_met_power_1'),
 (20, 'PRI_met_phi_power_1'),
 (21, 'PRI_met_sumet_power_1'),
 (22, 'PRI_jet_num_power_1'),
 (23, 'PRI_jet_leading_pt_power_1'),
 (24, 'PRI_jet_leading_eta_power_1'),
 (25, 'PRI_jet_leading_phi_power_1'),
 (26, 'PRI_jet_subleading_pt_power_1'),
 (27, 'PRI_jet_subleading_eta_power_1'),
 (28, 'PRI_jet_subleading_phi_power_1'),
 (29, 'PRI_jet_a

# Features

In [9]:
idx =  [1, 13, 4, 11, 46, 0, 44, 7, 12, 42, 49, 2, 16, 10, 43, 47, 45, 22, 6, 23, 19, 48, 21, 5, 8, 29, 32, 31, 30, 33]
X = X0[:,idx]
X.shape

(250000, 30)

# Set the model

In [10]:
initial_w = np.zeros(X.shape[1])
max_iters = 3000
gamma = 1e-1
t = 1e-4
w, loss = least_squares(Y,X)

In [11]:
y = predict_labels(w,X)

In [12]:
y.shape

(250000,)

In [13]:
pos = 0
neg = 0
for i in range (len(Y)):
    if Y[i] == y[i]:
        pos += 1
    else:
        neg += 1
print(pos)
print(neg)

success_rate = pos/(pos+neg)
print(success_rate)

170575
79425
0.6823


# Load the data test

In [14]:
nope, test_data, ids = load_csv_data(r'/home/ilaria/Scrivania/Machine_Learning/Project_1/test.csv', sub_sample=False)

print("Data loaded! Shape: ")
print(np.shape(test_data))

Data loaded! Shape: 
(568238, 30)


In [16]:
input_data, Y = handle_outliers(test_data,nope,-999,'mean')
ind_back, ind_sig = idx_2labels(Y, [-1,1])
Y[ind_back] = 0

# get feature names 
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
log = True
degree = 1
X0, features = extend_features(input_data, names, degree, log)
idx =  [1, 13, 4, 11, 46, 0, 44, 7, 12, 42, 49, 2, 16, 10, 43, 47, 45, 22, 6, 23, 19, 48, 21, 5, 8, 29, 32, 31, 30, 33]
X = X0[:,idx]

y_pred = predict_labels(w,X)

-999 are replaced by the mean value of the feature
---------------------------
Features have been set to the power(s): [1]
16 Features of the momentum have been added
4 logarithmic features have been added.
Data have been standardized.
---------------------------


In [17]:
y_pred.shape

(568238,)

In [18]:
y_pred[np.where(y_pred==0)] = -1

In [19]:
create_csv_submission(ids, y_pred, "a_sub")